# Components with hierarchy

You can define some components (waveguides, bends, couplers) as a stand alone components, with basic input parameters (width, length, radius ...)

Then you can re-use those components in a more complex hiearchical components.

`gdsfactory` does this by passing the higher level component with the lower level functions to build the components.

You can customize any of the functions thanks to `functools.partial`

In [ ]:
import pp

In [ ]:
@pp.cell
def mzi_sample(splitter=pp.c.mmi1x2, combiner=pp.c.mmi1x2, bend=pp.c.bend_euler, straight=pp.c.straight, delta_length:float=100.):
    arm_top_sequence = 'BR-RB'
    arm_top_symbol_to_component = {
        "B": (bend(), "W0", "N0"),
        "R": (bend(), "N0", "W0"),
        "-": (straight(), "W0", "E0"),
        "_": (straight(length=delta_length/2), "W0", "E0"),
    }
    arm_top_sequence = 'B_R-R_B'
    arm_bot_sequence = 'RB-BR'
    arm_top = pp.c.component_sequence(sequence=arm_top_sequence, symbol_to_component=arm_top_symbol_to_component)
    arm_bot = pp.c.component_sequence(sequence=arm_bot_sequence, symbol_to_component=arm_top_symbol_to_component)
    
    c = pp.Component()
    top = c << arm_top
    bot = c << arm_bot
    splitter =  c << splitter()
    combiner = c << combiner() 
    
    top.connect('in', splitter.ports['E1'])
    bot.connect('in', splitter.ports['E0'])
    combiner.connect('E1', bot.ports['out'])
    c.add_port('W0', port=splitter.ports['W0'])
    c.add_port('E0', port=combiner.ports['W0'])
    return c

c = mzi_sample()
c

Lets **customize** the functions that we pass.
For example, we want to increase the radius of the bend from the default 10um to 50um.

In [ ]:
pp.c.bend_circular()

## functools.partial

Partial lets you define different default parameters for a function, so you can modify the settings for the child cells.

In [ ]:
from functools import partial

In [ ]:
bend50 = partial(pp.c.bend_circular, radius=50)
bend50()

In [ ]:
type(bend50)

In [ ]:
bend50.func.__name__

In [ ]:
bend50.keywords

In [ ]:
c = mzi_sample(bend=partial(pp.c.bend_circular, radius=50))
c

In [ ]:
bend50(radius=10)

In [ ]:
c = mzi_sample()
c

## PDK custom fab

You can define a new PDK by creating function that customize partial parameters of the generic functions.

Lets say that this PDK uses layer (41, 0) for the pads (instead of the layer used in the generic pad function).

You can also access `functools.partial` from `pp.partial`

In [ ]:
import pp

pad = pp.partial(pp.c.pad, layer=(41, 0))

In [ ]:
pad()